In [1]:
#!git clone https://github.com/fchollet/keras.git && cd keras && python setup.py install --user

In [15]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
import numpy as np
import os

In [32]:
images = []
for filename in os.listdir('data/gui_and_text'):
    if filename[-3:] == "npz":
        image = np.load('data/gui_and_text/'+filename)
        images.append(image['features'])
images = np.array(images, dtype=float)
images = preprocess_input(images)

IR2 = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
features = IR2.predict(images)
print(features.shape)

(1500, 1536)


In [33]:
num_words = 19
tokenizer = Tokenizer(num_words=num_words, filters='', split="\n", lower=False)

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

X = []
for filename in os.listdir('data/gui_and_text'):
    if filename[-3:] == "gui":
        X.append(load_doc('data/gui_and_text/'+filename))
    
tokenizer.fit_on_texts([load_doc('data/syntax.vocab')])

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)
 
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)
        image_data.append(features[img_no])
        X.append(in_seq)
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

In [38]:
image_features = Input(shape=(1536,))
# image_features = Dense(1024, activation='relu')(image_features)
ir2_out = RepeatVector(max_length)(image_features)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 32, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

(?, 16, 32)
Tensor("lstm_2/TensorArrayReadV3:0", shape=(?, 512), dtype=float32)


In [ ]:
model.fit([image_data, X], y, batch_size=1, shuffle=False, epochs=5)

Epoch 1/5
11633/13376 [=========================>....] - ETA: 5:08 - loss: 3.0696e-04

In [1]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = '<START>'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += word
        # stop if we predict the end of the sequence
        if word == '<END>':
            break
    return in_text

In [3]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc in descriptions:
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        actual.append([desc.split()])
        predicted.append(yhat.split())
        print('Actual:    %s' % desc)
        print('Predicted: %s' % yhat)
        if len(actual) >= 5:
            break
    # calculate BLEU score
    bleu = corpus_bleu(actual, predicted)
    return bleu
evaluate_model(model, X[0], image_data[0], tokenizer, max_length)

NameError: name 'model' is not defined